# Web Scrapping Top 1000 movies IMDB

Web Scrapping Imports and code

In [4]:
#packages needed
import numpy as np
import pandas as pd
from requests import get
from bs4 import BeautifulSoup

#imported to add more human behavior
from time import sleep
from random import randint

#empty list containers 
metascores = []
us_gross = []
years = []
titles = []
imdb_ratings = []
actors = []
genres = []
directors = []
movie_ratings = []


#Setting range
pages = np.arange(1, 10, 50)

#Initial for loop to go through webpages set up with time delays
for page in pages:
    #setting each loop to be the url
    page = get('https://www.imdb.com/search/title/?groups=top_1000&start=" + str(page) + "&ref_=adv_nxt')
    #using soup to parse through each new url
    soup = BeautifulSoup(page.text, 'html.parser')
    #initial container for each movie
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    #created random intervals of waiting before next url is loaded
    sleep(randint(2,10))
    #loops through each container in each page
    for cont in movie_div:
        #setting each variable to push into the empty list
        movie_title = cont.h3.a.text
        titles.append(movie_title)
        
        movie_year = cont.h3.find('span', class_='lister-item-year').text
        years.append(movie_year)
        
        movie_rating = cont.p.find('span', class_='certificate').text
        movie_ratings.append(movie_rating)
        
        movie_genre = cont.p.find('span', class_='genre').text
        genres.append('movie_genre')
        
        imdb_rating = float(cont.strong.text)
        imdb_ratings.append(imdb_rating)
        
        metascore = cont.find('span', class_= 'metascore').text if container.find('span', class_='metascore') else ''
        metascores.append(metascore)
        
        cast_and_crew = cont.find_all('p', class_='').find_all('a')
        

In [73]:
    page = get('https://www.imdb.com/search/title/?groups=top_1000&start=51&ref_=adv_nxt')
    #using soup to parse through each new url
    soup = BeautifulSoup(page.text, 'html.parser')
    #initial container for each movie
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    cast_and_crew = movie_div[0].find_all('p', class_='')[0].find_all('a')[1:]
#     director = cast_and_crew[0]
cast_and_crew[0].text

'Roman Griffin Davis'